In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import yfinance as yf

In [2]:
#for testing cell

startDate = '2015-03-01'
endDate = '2017-03-01'

ticker = 'GOOGL'

resultData = yf.download(ticker, startDate, endDate)

resultData = resultData[["Open", "High","Low", "Close", "Adj Close", "Volume"]]


[*********************100%%**********************]  1 of 1 completed


In [3]:
train_main_series = resultData['Close']

In [4]:
kwargs_model = {'trend' : 'mul', 'damped_trend': True, 'initialization_method': 'estimated'}
horizon = 12

In [5]:
exp_sm = ExponentialSmoothing(
        train_main_series,
        **kwargs_model
    ).fit()

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [6]:
forecast_numpy = exp_sm.forecast(horizon).values

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [7]:
forecast_series = pd.Series(data = forecast_numpy, index=pd.date_range(train_main_series.index[-1], periods=horizon))

In [8]:
from supporting_forecast_fuctions import *

In [9]:
def evaluate_exp_sm(train_main_series, kwargs_model, steps = [60, 30, 14, 7]):
    metrics_df = pd.DataFrame(index = ['MAE', 'MAPE', 'RMSE'])

    for step in steps:
        fitted_model = ExponentialSmoothing(train_main_series[:-step], **kwargs_model).fit()
        forecast = fitted_model.forecast(step)
        metrics_df[step] = get_metrics(train_main_series[-step:], forecast).values()
    return metrics_df

In [10]:
evaluate_exp_sm(train_main_series, kwargs_model)

c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer ind

,60,30,14,7
MAE,1.607792,0.467379,0.492712,0.240310
MAPE,0.038758,0.011193,0.011642,0.005657
RMSE,1.740810,0.575925,0.567509,0.267729


In [11]:
from supporting_forecast_fuctions import plot_prediction


In [12]:
plot_prediction(train_main_series,forecast_series)